## **Notebook Objective**

This notebook explains **why the model makes its predictions** by:

* Ranking global feature importance
* Quantifying marginal contributions using SHAP
* Visualizing nonlinear and interaction effects
* Translating outputs into insurer- and policy-relevant insights

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.inspection import PartialDependenceDisplay
import shap
import joblib

pd.set_option("display.float_format", "{:.4f}".format)
plt.style.use("seaborn-v0_8")

In [ ]:
DATA_PATH = "../data/raw/insurance.csv"
PREPROCESSOR_PATH = "../data/processed/preprocessor.pkl"

df = pd.read_csv(DATA_PATH)
preprocessor = joblib.load(PREPROCESSOR_PATH)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

numerical_features = [
    "age", "age_squared", "bmi", "children", "smoker_bmi_interaction"
]

categorical_features = [
    "sex", "region", "bmi_category"
]

X = df[numerical_features + categorical_features]
y = np.log1p(df["charges"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

gb_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", GradientBoostingRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=3,
            random_state=42
        ))
    ]
)

gb_pipeline.fit(X_train, y_train)

In [ ]:
cat_features = preprocessor.named_transformers_["cat"] \
    .named_steps["onehot"] \
    .get_feature_names_out(categorical_features)

feature_names = numerical_features + list(cat_features)

In [ ]:
importances = gb_pipeline.named_steps["model"].feature_importances_

importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values("Importance", ascending=False)

importance_df.head(10)

In [ ]:
sns.barplot(
    data=importance_df.head(10),
    x="Importance",
    y="Feature"
)
plt.title("Top 10 Feature Importances")
plt.show()

In [ ]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
explainer = shap.TreeExplainer(gb_pipeline.named_steps["model"])
shap_values = explainer.shap_values(X_test_processed)

In [ ]:
shap.summary_plot(
    shap_values,
    X_test_processed,
    feature_names=feature_names,
    plot_type="bar"
)

In [ ]:
shap.summary_plot(
    shap_values,
    X_test_processed,
    feature_names=feature_names
)

In [ ]:
sample_idx = 5

shap.force_plot(
    explainer.expected_value,
    shap_values[sample_idx],
    X_test_processed[sample_idx],
    feature_names=feature_names,
    matplotlib=True
)

In [ ]:
PartialDependenceDisplay.from_estimator(
    gb_pipeline,
    X_train,
    ["age"],
    grid_resolution=50
)
plt.show()

In [ ]:
PartialDependenceDisplay.from_estimator(
    gb_pipeline,
    X_train,
    ["smoker_bmi_interaction"],
    grid_resolution=50
)
plt.show()

In [ ]:
interpretation_summary = {
    "Top Cost Driver": "Smoking status",
    "Second Order Drivers": "BMI, Age",
    "Nonlinear Effects Captured": "Yes",
    "Interaction Effects": "Smoker × BMI",
    "Model Explainability": "High (SHAP-supported)"
}

pd.DataFrame.from_dict(
    interpretation_summary,
    orient="index",
    columns=["Finding"]
)

## **Key Conclusions**

* Model decisions are **clinically and economically intuitive**
* Smoking status creates the largest marginal cost uplift
* BMI shows threshold-driven cost increases
* Age effects accelerate at higher values
* Model is suitable for **regulated, explainable use cases**